In [ ]:
import networkx as nx
from EXP3 import EXP3, compute_regret, plot_regret
import arms
import numpy as np
import obsGraph

In [ ]:
G = nx.DiGraph()
G.add_node(0, arm=arms.ArmBernoulli(0.5))
G.add_node(1, arm=arms.ArmBernoulli(0.3))
G.add_node(2, arm=arms.ArmBernoulli(0.4))
G.add_node(3, arm=arms.ArmBernoulli(0.7))
G.add_edge(0,0)
G.add_edge(0,1)
G.add_edge(2,2)
G.add_edge(3,3)

G = nx.convert_node_labels_to_integers(G)

In [ ]:
import matplotlib.pyplot as plt
nx.draw(G)  # networkx draw()
plt.draw()
plt.show()

In [ ]:
n_itr = 10000
q,losses = EXP3(G, list(G.nodes()), 0.5, 0.05, n_itr)

In [ ]:
regret = compute_regret(losses, G)

In [ ]:
plot_regret([regret], ['EXP3'])

In [ ]:
alpha = 1/2
beta = 1/2
n_nodes = 20
H = obsGraph.strong_obs_graph(n_nodes, alpha, beta)

In [ ]:
nx.draw(H)  # networkx draw()
plt.draw()
plt.show()

In [ ]:
obsGraph.observability_type(H)

In [ ]:
obsGraph.strong_nodes(H)

# Examples
## Strongly observable
### Bandit

In [ ]:
bandit = nx.DiGraph()
bandit.add_node(0, arm=arms.ArmBernoulli(0.5))
bandit.add_node(1, arm=arms.ArmBernoulli(0.3))
bandit.add_node(2, arm=arms.ArmBernoulli(0.4))
bandit.add_node(3, arm=arms.ArmBernoulli(0.7))
bandit.add_node(4, arm=arms.ArmBernoulli(0.1))
bandit.add_edge(0,0)
bandit.add_edge(1,1)
bandit.add_edge(2,2)
bandit.add_edge(3,3)
bandit.add_edge(4,4)
print(obsGraph.observability_type(bandit))

In [ ]:
nx.draw(bandit)  # networkx draw()
plt.draw()
plt.show()

In [ ]:
n_itr = 10000
bandit_q, bandit_losses = EXP3(bandit, list(bandit.nodes()), 0.5, 0.05, n_itr)
bandit_regret = compute_regret(bandit_losses, bandit)

In [ ]:
plot_regret([bandit_regret], ['EXP3'])

In [ ]:
bandit_q

independent set np hard
https://networkx.github.io/documentation/networkx-1.10/reference/algorithms.approximation.html?highlight=independent%20set#module-networkx.algorithms.approximation.independent_set

how to compute weak domination number?

regret doit être une esperance: il faut lancer plusieurs fois et faire la moyenne

### Full feedback

In [ ]:
graph_arms = [arms.ArmBernoulli(0.5), arms.ArmBernoulli(0.3), arms.ArmBernoulli(0.4), arms.ArmBernoulli(0.7), arms.ArmBernoulli(0.1)]
full_feedback = obsGraph.strong_obs_graph(5, 0, 0, graph_arms)
print(obsGraph.observability_type(full_feedback))

In [ ]:
nx.draw(full_feedback)  # networkx draw()
plt.draw()
plt.show()

In [ ]:
n_itr = 10000
full_feedback_q, full_feedback_losses = EXP3(full_feedback, list(full_feedback.nodes()), 0.5, 0.05, n_itr)
full_feedback_regret = compute_regret(full_feedback_losses, full_feedback)

In [ ]:
plot_regret([full_feedback_regret], ['EXP3'])

In [ ]:
plot_regret([full_feedback_regret, bandit_regret], ['Full feedback', 'Bandit'])

### Police officer - loopless clique

In [ ]:
graph_arms = [arms.ArmBernoulli(0.5), arms.ArmBernoulli(0.3), arms.ArmBernoulli(0.4), arms.ArmBernoulli(0.7), arms.ArmBernoulli(0.1)]
police = obsGraph.strong_obs_graph(5, 1, 0, graph_arms)
print(obsGraph.observability_type(police))

In [ ]:
nx.draw(police)  # networkx draw()
plt.draw()
plt.show()

In [ ]:
n_itr = 10000
police_q, police_losses = EXP3(police, list(police.nodes()), 0.5, 0.05, n_itr)
police_regret = compute_regret(police_losses, police)

In [ ]:
plot_regret([police_regret], ['EXP3'])

In [ ]:
plot_regret([full_feedback_regret, bandit_regret, police_regret], ['Full feedback', 'Bandit', 'Police'])

## Weakly observabnle

### Revealing actions

In [ ]:
revealing = nx.DiGraph()
revealing.add_node(0, arm=arms.ArmBernoulli(0.5))
revealing.add_node(1, arm=arms.ArmBernoulli(0.3))
revealing.add_node(2, arm=arms.ArmBernoulli(0.4))
revealing.add_node(3, arm=arms.ArmBernoulli(0.7))
revealing.add_node(4, arm=arms.ArmBernoulli(0.1))
revealing.add_edge(0,0)
revealing.add_edge(0,1)
revealing.add_edge(0,2)
revealing.add_edge(0,3)
revealing.add_edge(0,4)
print(obsGraph.observability_type(revealing))

In [ ]:
nx.draw(revealing)  # networkx draw()
plt.draw()
plt.show()

In [ ]:
n_itr = 10000
revealing_q, revealing_losses = EXP3(revealing, list(revealing.nodes()), 0.5, 0.05, n_itr)
revealing_regret = compute_regret(revealing_losses, revealing)

In [ ]:
plot_regret([revealing_regret], ['EXP3'])

In [ ]:
plot_regret([full_feedback_regret, bandit_regret, police_regret, revealing_regret], ['Full feedback', 'Bandit', 'Police','Revealing'])

## Unobservable

# Instability

## Strongly to weakly

## Strongly to unobservable

## Weakly to unobservable